In [1]:
!pip install -q parlai

     |████████████████████████████████| 1.7 MB 5.6 MB/s 
     |████████████████████████████████| 248 kB 47.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 64 kB 974 kB/s 
     |████████████████████████████████| 547 kB 35.9 MB/s 
     |████████████████████████████████| 95 kB 3.9 MB/s 
     |████████████████████████████████| 8.7 MB 31.6 MB/s 
     |████████████████████████████████| 56 kB 2.1 MB/s 
     |████████████████████████████████| 125 kB 34.7 MB/s 
     |████████████████████████████████| 132 kB 11.0 MB/s 
     |████████████████████████████████| 15.7 MB 269 kB/s 
     |████████████████████████████████| 2.8 MB 36.5 MB/s 
     |████████████████████████████████| 175 kB 39.4 MB/s 
     |████████████████████████████████| 138 kB 38.7 MB/s 
     |████████████████████████████████| 145 kB 43.9 MB/s 
     |████████████████████

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/elomia

Mounted at /content/drive
/content/drive/MyDrive/elomia


In [6]:
import pandas as pd
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

emd_df = pd.read_csv("API/ParlAI_ex/emp_data_train.csv", on_bad_lines='skip')
emd_df.reset_index(inplace=True)

In [7]:
import numpy as np

slice_most_repeated_indexes = emd_df['speaker_idx'].value_counts()[0:20].index

most_repeated = emd_df[emd_df['speaker_idx'].isin(slice_most_repeated_indexes)]

most_repeated["two_input"] = most_repeated[['utterance', 'prompt']].apply(lambda x: ' '.join(x), axis=1)

unq_speakers = most_repeated["speaker_idx"].unique().tolist()
print(len(unq_speakers))
most_repeated["speaker_label"] = list(map(lambda x: unq_speakers.index(x), most_repeated["speaker_idx"]))

unq_contexts = most_repeated["context"].unique().tolist()
print(len(unq_contexts))

most_repeated["context_label"] = list(map(lambda x: unq_contexts.index(x), most_repeated["context"]))

20
32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [8]:
train_data = most_repeated[["two_input", "speaker_label", "context_label"]]
print(len(train_data["two_input"]))
print(len(train_data[["speaker_label", "context_label"]]))

27482
27482


#PD->Torch

In [23]:
import re 

from torchtext.legacy import data

import torch 
import numpy as np 
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import TensorDataset, DataLoader

from torch.nn.utils.rnn import pad_sequence

tokenizer = get_tokenizer('basic_english')

# pad_dataset = TensorDataset(x_padded)

# def cleanup_text(texts):
#     cleaned_text = []
#     for text in texts:
#         # remove punctuation
#         text = re.sub('[^a-zA-Z0-9]', ' ', text)
#         # remove multiple spaces
#         text = re.sub(r' +', ' ', text)
#         # remove newline
#         text = re.sub(r'\n', ' ', text)
#         cleaned_text.append(text)
#     return cleaned_text




def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

all_text = train_data["two_input"]
all_text_one_str = list(map(lambda x: ' '.join(x), all_text.values))
vocab = build_vocab_from_iterator(yield_tokens(all_text_one_str), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])     



# prompt_items = list(map(lambda x: torch.Tensor(vocab(x.split(" "))), np.array(prompts)))
# padded_prompt = pad_sequence(prompt_items)

# print(prompt_items[0])
# print(padded_prompt[0])


# pad_dataset = TensorDataset(x_padded)

# # Create random batches
# dataset_loader = DataLoader(pad_dataset, batch_size=32, shuffle=True)
# print(len(dataset_loader))
# t_i = next(iter(dataset_loader))[0]
# print(t_i.shape)

In [24]:
import torch
from torch import nn
import torch.nn.functional as F

class MultiOutputNetwork(nn.Module):
    def __init__(self, input_len, num_of_speakers, num_of_emotions): # <- input_len it's bad
        super().__init__()

        hidden_size = 256
        self.embeddings = nn.Embedding(input_len, hidden_size)

        self.branch_1 = nn.Linear(hidden_size, num_of_speakers) 

        self.branch_2 = nn.Linear(hidden_size, num_of_emotions) 
        
    def forward(self, x_1, x_2):
        x_1 = self.embeddings(x_1)
        x_2 = self.embeddings(x_2)

        out_1 = F.softmax(self.branch_1(x_1))

        out_2 = F.softmax(self.branch_2(x_2))
        
        return out_1, out_2

# Test model

In [25]:
import random

model = MultiOutputNetwork(input_len=784, num_of_speakers = 10, num_of_emotions = 10)
input = [random.randint(0,10) for i in range(784)] # Hmmmm
input = torch.FloatTensor(input)
input = input.expand([1, 784])
print(input.shape)

preds = model(input.to(torch.int64), input.to(torch.int64)) # Hmmmmm
print(preds)

torch.Size([1, 784])
(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], grad_fn=<SoftmaxBackward0>), tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], grad_fn=<SoftmaxBackward0>))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
from parlai.core.agents import Agent
from parlai.core.params import ParlaiParser
from parlai.core.worlds import create_task
from parlai.core.opt import Opt

In [27]:
class PersonEvaluatorAgent(Agent):
    # initialize by setting id
    def __init__(self):
        self.id = 'Dump agent'
        self.model = MultiOutputNetwork(input_len=len(vocab), num_of_speakers = 10, num_of_emotions = 10)


    def observe(self, observation):
        self.observation = observation

        item = observation["prompt"].split(" ")
        prompt = torch.IntTensor(vocab(item))

        item = observation["utterance"].split(" ")
        utterance = torch.IntTensor(vocab(item))

        observation["preds"] = self.model(prompt, utterance)
        return observation

    def act(self):
        reply = {'id': self.id}
        reply["preds"] = self.observation["preds"]
        return reply

In [28]:
from parlai.core.teachers import register_teacher, DialogTeacher, Teacher#, EmpatheticDialoguesTeacher
from parlai.core.message import Message
# import pandas as pd

# from parlai.tasks.empathetic_dialogues.agents import EmpatheticDialoguesTeacher
   

@register_teacher("Dump teacher")
class MyTeacher(Teacher):
    def __init__(self, opt, shared=None):

        opt['datafile'] = "API/ParlAI_ex/emp_data_train.csv"

        self.prompts = iter(emd_df["prompt"].tolist())
        self.utterances = iter(emd_df["utterance"].tolist())
        self.speaker_ids = iter(emd_df["speaker_idx"].tolist())
        self.contexts = iter(emd_df["context"].tolist())

        self.observation = {}
        super().__init__(opt, shared)

    def observe(self, reply):
        print("reply preds" , reply["preds"])
      
    def act(self):
        observation = {}
        observation["prompt"] = next(self.prompts)
        observation["utterance"] = next(self.utterances)
        observation["speaker_idx"] = next(self.speaker_ids)
        observation["context"] = next(self.contexts)

        return observation


In [29]:
opt = Opt()
opt['task']= "Dump teacher"#"Dump teacher"
opt['datatype']="train"

teacher = MyTeacher(opt)

agent = PersonEvaluatorAgent()
world = create_task(opt, agent)

for _ in range(10):
    world.parley()
    for a in world.acts:
    # print the actions from each agent
        print("acts", a)

    print(world.display())
    if world.epoch_done():
        print('EPOCH DONE')
        break

18:17:20 | creating task(s): Dump teacher
reply preds (tensor([[0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.0876, 0.0626, 0.0808, 0.1188,
         0.0631],
        [0.2175, 0.0449, 0.1012, 0.0502, 0.1734, 0.087

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
